In [1]:
import os, glob
import numpy as np

In [2]:
cwd = os.getcwd()
cwd

'/Users/bw/Code/lightning/src'

In [3]:
val_path = '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input'
print(val_path)

/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input


In [4]:
path = os.path.join(val_path, '*.bin' )
print(path)
valfiles = glob.glob(path, recursive=True)
valfiles.sort()
valfiles

/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/*.bin


['/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0801.bin',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0802.bin',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0803.bin',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0804.bin',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0805.bin',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0806.bin',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0807.bin',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0808.bin',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0809.bin',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0810.bin',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0811.bin',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0812.bin',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0813.bin',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0814.bin',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0815.bin',
 '/Users/bw/Dataset/MIPI_

In [5]:
def read_bin(img_path):
    """
    Read bin data
    """
    img_data = np.fromfile(img_path, dtype=np.uint16)
    w = int(img_data[0])
    h = int(img_data[1])
    assert w * h == img_data.size - 2
    quad = np.clip(img_data[2:].reshape([h, w]).astype(np.float32), 0, 1023)
    return quad

In [8]:
for idx, file in enumerate(valfiles):
    

    
    input_quad = read_bin(file)

    
    
    
    fname_npy = file[:-4] + '.npy'
    print(fname_npy)
    np.save(fname_npy, input_quad)
    
#     break

/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0801.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0802.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0803.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0804.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0805.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0806.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0807.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0808.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0809.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0810.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0811.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0812.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0813.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0814.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0815.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0816.npy
/Users/bw/Dataset/MIPI_d

# Cure BP's

In [9]:
def cure_static_bp(arr_patch):
    ## Red
    for yy in range(1, arr_patch.shape[0], 4):
        for xx in range(1, arr_patch.shape[1], 4):
            arr_patch[yy][xx] = ((arr_patch[yy - 1][xx] + arr_patch[yy][xx - 1]) / 2)

    ## Blue
    for yy in range(3, arr_patch.shape[0], 4):
        for xx in range(3, arr_patch.shape[1], 4):
            arr_patch[yy][xx] = ((arr_patch[yy - 1][xx] + arr_patch[yy][xx - 1]) / 2)
    return arr_patch




In [10]:

def cure_dynamic_bp(arr_patch, nbp=4):
    #        Quad                idx_R
    #
    #   R R G G R R G G  |  O O X X O O X X
    #   R R G G R R G G  |  O O X X O O X X
    #   G G B B G G B B  |  X X O O X X O O
    #   G G B B G G B B  |  X X O O X X O O
    #   R R G G R R G G  |  O O X X O O X X
    #   R R G G R R G G  |  O O X X O O X X
    #   G G B B G G B B  |  X X O O X X O O
    #   G G B B G G B B  |  X X O O X X O O


    # print('-->', idx_R>0)
    for yy in range(0, arr_patch.shape[0]-8, 2):
        for xx in range(0, arr_patch.shape[1]-8, 2):
            sy = yy
            ey = sy+8
            sx = xx
            ex = sx+8
            patch = arr_patch[sy:ey, sx:ex]

            
            for _ in range(nbp):
                red = patch[idx_R > 0]
                red.sort()
                median = int(np.median(red))
                if np.abs(red[-1] - red[-2]) > (64/1023):
                    patch[(patch == red[-1]) & (idx_R > 0)] = median

            arr_patch[sy:ey, sx:ex] = patch

    return arr_patch

In [12]:
val_path = '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val'
path = os.path.join(val_path, 'input/*.npy' )
print(path)
valfiles = glob.glob(path, recursive=True)
valfiles.sort()
valfiles

/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/*.npy


['/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0801.npy',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0802.npy',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0803.npy',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0804.npy',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0805.npy',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0806.npy',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0807.npy',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0808.npy',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0809.npy',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0810.npy',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0811.npy',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0812.npy',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0813.npy',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0814.npy',
 '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0815.npy',
 '/Users/bw/Dataset/MIPI_

In [20]:
cure_path = '/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input_cure'

In [24]:
for idx, file in enumerate(valfiles):
    print("%02d, "%(idx+1), file)
    npyname = file.split('/')[-1]
    print(npyname)
    npyname = os.path.join(cure_path, npyname)
    print(npyname)
    
    arr = np.load(file)
    
    arr = cure_static_bp(arr)
    arr = cure_static_bp(arr)
    
    np.save(npyname, arr)
#     break
    
    

01,  /Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0801.npy
0801.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input_cure/0801.npy
02,  /Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0802.npy
0802.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input_cure/0802.npy
03,  /Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0803.npy
0803.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input_cure/0803.npy
04,  /Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0804.npy
0804.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input_cure/0804.npy
05,  /Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0805.npy
0805.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input_cure/0805.npy
06,  /Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0806.npy
0806.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input_cure/0806.npy
07,  /Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input/0807.npy
0807.npy
/Users/bw/Dataset/MIPI_demosaic_hybridevs/val/input_cure/0807.npy
08,  /Users/b